# SVG

In [38]:
import base64
import drawSvg as draw
from IPython.display import display, HTML
from io import StringIO

blue = "rgb(83,160,227)"
trans = "none"

class Icon:
    def __init__(self, width, height, stroke, stroke_width):
        self.width = width
        self.height = height
        self.stroke = stroke
        self.stroke_width = stroke_width
        self.d = draw.Drawing(width, height)
        
    def add_rect(self, x, y, w, h, fill="none", stroke=None, stroke_width=None):
        stroke = stroke or self.stroke
        stroke_width = stroke_width or self.stroke_width
        self.d.append(
            draw.Rectangle(x, y, w, h, fill=fill, stroke=stroke, stroke_width=stroke_width, stroke_linejoin="round")
        )
        
    def add_lines(self, *points, fill="none", close=True, stroke=None, stroke_width=None, stroke_linejoin="round", stroke_linecap="round"):
        stroke = stroke or self.stroke
        stroke_width = stroke_width or self.stroke_width
        self.d.append(
            draw.Lines(*points, close=close, fill=fill, stroke=stroke, stroke_width=stroke_width, stroke_linejoin=stroke_linejoin, stroke_linecap=stroke_linecap)
        )
        
    def add_arc(self, cx, cy, r, startDeg, endDeg, fill="none", stroke=None, stroke_width=None):
        stroke = stroke or self.stroke
        stroke_width = stroke_width or self.stroke_width
        self.d.append(
            draw.Arc(cx, cy, r, startDeg, endDeg, cw=False, stroke=stroke, stroke_width=stroke_width, fill=fill, stroke_linecap="round")
        )

    def add_ellipse(self, cx, cy, rx, ry, fill="none", stroke=None, stroke_width=None):
        stroke = stroke or self.stroke
        stroke_width = stroke_width or self.stroke_width
        self.d.append(
            draw.Ellipse(cx, cy, rx, ry, fill=fill, stroke=stroke, stroke_width=stroke_width)
        )

    def as_svg(self, name, width, height):
        outputFile = StringIO()
        outputFile.write('''<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" width="{}" height="{}" viewBox="{} {} {} {}">'''.format(
                width, height, *self.d.viewBox))
        outputFile.write('\n')

        isDuplicate = lambda x: False

        for element in self.d.allElements():
            element.writeSvgElement(name, isDuplicate, outputFile, False)
            outputFile.write('\n')
        outputFile.write('</svg>\n')

        return outputFile.getvalue()


class Icons:
    def __init__(self, width, height, background):
        self.width = width
        self.height = height
        self.background = background
        self.icons = {}
        
    def append(self, name, icon):
        self.icons[name] = icon
        
    def show(self, theme, kind, show_html=False):
        html = f"""<style>
    .btn_{kind}_{theme} {{
        display: inline-block;
        width: {self.width}px;
        height: {self.height}px;
        background-color: {self.background}; 
        padding: 0px;
        margin: 4px;
        border: solid 1px red;
    }}
    .all_{theme} {{
        background-color: {self.background};     
    }}
    </style>"""

        for name, icon in self.icons.items():
            html += f'<div class="btn_{kind}_{theme}">{icon.as_svg(name, self.width, self.height)}</div>\n'
        if show_html:
            print(html)
        else:
            display(HTML(f"""<div class="all_{theme}">{html}</div>"""))

    def save(self):
        for name, icon in self.icons.items():
            xml = icon.as_svg(name, width, height)
            b64 = base64.b64encode(bytes(xml, "utf-8")).decode("utf-8")
            with open(f"{name}.svg", "w") as fd:
                fd.write(xml)

            with open(f"{name}.b64", "w") as fd:
                fd.write(b64) 

In [60]:
def cube(name, width, height, margin, stroke, stroke_width,
         ftop="none", fbottom="none", ffront="none", frear="none", fright="none", fleft="none"):
    m = margin
    r = 2 * (width - m) / 3   # rect size
    o = r / 2                 # offset
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(o, o, r,  r, fill=frear)
    icon.add_lines(m,m, m,m+r, o,o+r, o,o, fill=fleft)
    icon.add_lines(m,m, o,o, o+r,o, r+m,m, fill=fbottom)
    icon.add_lines(m,m+r, o,o+r, o+r,o+r, r+m,m+r, fill=ftop)
    icon.add_lines(m+r,m, m+r,m+r, o+r,o+r, o+r,o, fill=fright)
    icon.add_rect(m, m, r, r, fill=ffront)
    return icon

def iso(width, height, margin, stroke, stroke_width):
    x2 = 2 * margin
    x1 = width / 2
    x0 = width - 2 * margin
    y4 = margin
    y3 = margin + (width - 2 * margin) / 4
    y2 = margin + (width - 2 * margin) / 2
    y1 = margin + (width - 2 * margin) / 4 * 3
    y0 = width - margin
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(x0,y3, x0,y1, x1,y2, x1,y4, fill=blue)
    icon.add_lines(x1,y2, x1,y4, x2,y3, x2,y1, fill=blue)
    icon.add_lines(x0,y1, x1,y0, x2,y1, x1,y2, fill=blue)
    return icon
   
def reset(width, height, margin, stroke, stroke_width):
    cx = cy = width / 2
    r1 = width / 2  - 2 * margin
    r2 = r1 - 2 * margin
    r = r1 - margin
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_arc(cx, cy, r1, 60, 360)
    icon.add_arc(cx, cy, r2, 60, 360)
    icon.add_lines(
        cx + r2 - margin, cy, 
        cx + r2 + margin, cy + 2 * margin,
        cx + r1 + margin, cy,
        close=False
    )
    return icon

def resize(width, height, margin, stroke, stroke_width):
    m = margin
    r = width - 2 * m
    d = 2 * m
    md = 3 * m
    mo = 5 * m
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(m, m, r, r)
    icon.add_lines(md, md, mo, mo)
    icon.add_lines(width - md, height - md, width - mo, height - mo)
    icon.add_lines(width - md, md, width - mo, mo)
    icon.add_lines(md, height - md, mo, height - mo)
    icon.add_lines(md, md, md, md + d)
    icon.add_lines(md, md, md + d, md)
    icon.add_lines(width - md, height - md, width - md, height - md - d)
    icon.add_lines(width - md, height - md, width - md - d, height - md)
    icon.add_lines(md, height - md, md, height - md - d)
    icon.add_lines(md, height - md, md + d, height-md)
    icon.add_lines(width - md, md, width-md, md + d)
    icon.add_lines(width - md, md, width - md - d, md)
    return icon

def plane(width, height, margin, stroke, stroke_width):
    m = margin
    x0 = (margin, 2.5 * margin)
    x1 = (2.2 * margin, width / 2)
    x2 = (width - margin, width / 2)
    x3 = (width - 2.5 * margin, 2.5 * margin)
    c = width / 2
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(*x0, *x1, *x2, *x3)
    icon.add_lines(c,  1.5 * m + height/4, c, c + height/4, stroke=blue, stroke_width=1.5*stroke_width)
    icon.add_lines(c - m, c + height/4 - m, c, c + height/4, c + m, c + height/4 - m, stroke=blue, stroke_width=1.5*stroke_width)
    
    return icon

def nav_icons(factor, theme="light"):
    if theme == "light":
        stroke = "#666"
        background = "#fff"
    else:
        stroke = "#ccc"
        background = "#444"
        
    stroke_width = factor / 5
    margin = factor / 4
    width = height = factor * 3 + margin

    icons = Icons(width, height, background)

    for name in ["top", "bottom", "front", "rear", "left", "right"]:
        icon = cube(name, width, height, margin, stroke, stroke_width, **{f"f{name}": blue})
        icons.append(name, icon)

    icons.append("iso", iso(width, height, margin, stroke, stroke_width))
    icons.append("reset", reset(width, height, margin, stroke, stroke_width))
    icons.append("resize", resize(width, height, margin, stroke, stroke_width))
    icons.append("plane", plane(width, height, margin, stroke, stroke_width))

    return icons

In [61]:
def stop(width, height, margin, stroke, stroke_width):
    m = margin

    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(1.4*m, 1.4*m, width - 2.8*m, width - 2.8*m)
    icon.add_rect(3*m, 3*m, width - 6*m, width - 6*m, fill=blue)
    return icon

def pause(width, height, margin, stroke, stroke_width):
    m = margin

    icon = Icon(width, height, stroke, stroke_width)
    icon.add_rect(3.3*m, 1.4*m, 2.2*m, width - 2.8*m, fill=blue)
    icon.add_rect(width-5.4*m, 1.4*m, 2.2*m, width - 2.8*m, fill=blue)
    return icon
    
def play(width, height, margin, stroke, stroke_width):
    m = margin
    
    icon = Icon(width, height, stroke, stroke_width)
    icon.add_lines(1.8*m, 1.4*m, 1.8*m, height - 1.4*m, width - 1.8*m, width/2)
    icon.add_lines(3.3*m, 4*m, 3.3*m, height - 4*m, width - 5*m, width/2, fill=blue)
    
    return icon

def ani_icons(factor, theme="light"):
    if theme == "light":
        stroke = "#666"
        background = "#fff"
    else:
        stroke = "#ccc"
        background = "#444"
        
    stroke_width = factor / 5
    margin = factor / 4
    width = height = factor * 3 + margin

    icons = Icons(width, height, background)
    
    icons.append("play", play(width, height, margin, stroke, stroke_width))
    icons.append("pause", pause(width, height, margin, stroke, stroke_width))
    icons.append("stop", stop(width, height, margin, stroke, stroke_width))
    
    return icons

ani_icons(64).show("light", "ani")

In [62]:
def mesh(width, height, margin, stroke, stroke_width, mix=False, empty=False, no=False):
    m = margin
    dx = (width - 4 * margin) / 2
    dy = (height - 2 * margin) / 3
    d =  m
    x0 = margin
    x1 = margin + dx
    x2 = margin + 2 * dx
    y0 = margin
    y1 = margin + dy
    y2 = margin + 2 * dy
    y3 = margin + 3 * dy
    o = 2 * margin
    
    if empty:
        stroke = blue
    if no:
        fill = stroke
    else:
        fill = blue
    icon = Icon(width, height, stroke, stroke_width)
    if not mix:
        icon.add_lines(
            x0 + o, y0, x0, y1, x0, y2, x0 + o, y3,
            x1 + o, y3, x1, y2, x1, y1, x1 + o, y0,
            fill=fill, stroke=stroke, stroke_linecap="square", stroke_linejoin="miter"
        )
        icon.add_lines(
            x1 + o, y0, x1, y1, x1, y2, x1 + o, y3,
            x2 + o, y3, x2, y2, x2, y1, x2 + o, y0,
            fill=fill, stroke=stroke, stroke_linecap="square", stroke_linejoin="miter"
        )
        if not empty:
            icon.add_lines(
                x0, y1, x0, y2, x1, y2, x2, y2, x2, y1, stroke_linecap="square", stroke_linejoin="round"
            )
    else:
        icon.add_lines(
            x0 + o, y3, x1 + o - d, y3, x1 - d, y2, x1 - d, y1, x1 + o - d, y0, 
            x0 + o, y0, x0, y1, x0, y2, stroke=blue, fill=blue, stroke_linejoin="miter"
        )            
        icon.add_lines(
            x1 + o + d, y3, x2 + o, y3, x2, y2, x2, y1, x2 + o, y0, 
            x1 + o + d, y0, x1 + d, y1, x1 + d, y2, stroke=blue, fill=blue, stroke_linejoin="miter"
        )            
        icon.add_lines(
            x1, y2, x0, y2, x0, y1, x1, y1, close=False
        )
        icon.add_lines(
            x1 + o, y3, x0 + o, y3, x0, y2, x0, y1, x0 + o, y0, x1 + o, y0, close=False
        )
           
    return icon

def shape(width, height, margin, stroke, stroke_width, mix=False, empty=False, no=False):
    m = margin
    cx = width / 2
    cy = height / 2
    r = (width - 2*m) / 2
    r2 = width / 4.2
    r3 = width / 12
    
    e1 = 26
    e2 = 180 - e1
    e3 = 90 if mix else e1

    icon = Icon(width, height, stroke, stroke_width)
    if not empty:
        if mix:
            icon.add_arc(cx, cy, r2, 90, 270, fill=blue, stroke=trans, stroke_width=stroke_width)
            icon.add_arc(cx, cy, r3, 90, 270, fill=stroke)    
            icon.add_arc(cx, cy-2.5*m, r, e1, e2/2-m, stroke_width=stroke_width)
            icon.add_arc(cx, cy+2.5*m, r, m-e2/2, -e1, stroke_width=stroke_width)
        else:
            icon.add_ellipse(cx, cy, r2, r2, fill=stroke if no else blue, stroke_width=stroke_width, stroke=trans)
            if not no:
                icon.add_ellipse(cx, cy, r3, r3, fill=stroke)    
    icon.add_arc(cx, cy-2.5*m, r, e3, e2, fill=trans, stroke_width=stroke_width)
    icon.add_arc(cx, cy+2.5*m, r, -e2, -e3, fill=trans, stroke_width=stroke_width)
    return icon

def tree_icons(factor, theme="light"):
    if theme == "light":
        stroke = "#666"
        background = "#fff"
        missing = "#ddd"
    else:
        stroke = "#ccc"
        background = "#444"
        missing = "#666"
    
    stroke_width = factor / 5
    margin = factor / 4
    width = factor / 8 * 26 + margin
    height = factor / 8 * 16 + margin
    print(factor, width, height)
    icons = Icons(width, height, background)

    icons.append("shape", shape(width, height, margin, stroke, stroke_width))
    icons.append("shape_mix", shape(width, height, margin,stroke, stroke_width, mix=True))
    icons.append("shape_no", shape(width, height, margin,stroke, stroke_width, empty=True))
    icons.append("shape_empty", shape(width, height, margin, missing, stroke_width, no=True))

    icons.append("mesh", mesh(width, height, margin, stroke, stroke_width))
    icons.append("mesh_mix", mesh(width, height, margin, stroke, stroke_width, mix=True))
    icons.append("mesh_no", mesh(width, height, margin, stroke, stroke_width, empty=True))
    icons.append("mesh_empty", mesh(width, height, margin, missing, stroke_width, no=True))

    return icons

icons = tree_icons(factor=8, theme="light")

8 28.0 18.0


## Show icons

In [63]:
factor = 8

for theme in ["light", "dark"]:

    icons = nav_icons(factor=factor, theme=theme)
    icons.show(theme, "nav")

    icons = ani_icons(factor=factor, theme=theme)
    icons.show(theme, "ani")
    
    icons = tree_icons(factor=factor, theme=theme)
    icons.show(theme, "tree")
    


8 28.0 18.0


8 28.0 18.0


## Install icons

In [64]:
import os
import base64

template = """.btn_{mode}_{name} {{
  background:
    url(data:image/svg+xml;base64,{b64})
    no-repeat
    center center;
}}
"""

with open("icons.css", "a") as fd:
    for theme in  ["light", "dark"]:
        for icons in [nav_icons(factor=factor, theme=theme), tree_icons(factor=factor, theme=theme), ani_icons(factor=factor, theme=theme)]:
            for name, icon in icons.icons.items():
                data = icon.as_svg(name, icon.width, icon.height)
                b64 = base64.b64encode(data.encode()).decode()
                fd.write(template.format(name=name, mode=theme, b64=b64))
        

8 28.0 18.0
8 28.0 18.0
